# Lab 5: Deep Research

One of the classic cross-business Agentic use cases!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
# Imports

from agents import Agent, WebSearchTool, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import os
from IPython.display import display, Markdown
from pprint import pprint
import requests
load_dotenv(override=True)

# Constants

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following built-in tools:

- `WebSearchTool` lets an agent search the web.
- `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.
- `ComputerTool` allows automating computer use tasks.
- `CodeInterpreterTool` lets the LLM execute code in a sandboxed environment.
- `HostedMCPTool` exposes a remote MCP server's tools to the model.
- `ImageGenerationTool` generates images from a prompt.
- `LocalShellTool` runs shell commands on your machine.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool..

Costs are here: https://platform.openai.com/docs/pricing#web-search

## We will be making 4 Agents:

1. Search Agent - searches online given a search term using an OpenAI hosted tool
2. Planner Agent - given a query from the user, come up with searches
3. Report Agent - make a report on results
4. Push Agent - send a notification to the user's phone with a summary

## Our First Agent: Search Agent

Given a Search term, search for it on the internet and summarize results.

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [4]:
message = "What are the most popular and successful AI Agent frameworks in May 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of May 2025, several AI agent frameworks have gained prominence for their capabilities and developer adoption:

- **LangChain**: A modular Python framework for building LLM-powered agents, supporting various large language models and offering tools for chaining prompts, models, memory, and external tools. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

- **AutoGen**: Developed by Microsoft, AutoGen facilitates multi-agent systems and code automation, enabling agents to communicate in natural language to complete tasks. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

- **CrewAI**: Orchestrates role-playing AI agents for collaborative tasks, focusing on simplicity and minimal setup, making it popular for customer service and marketing automation. ([datacamp.com](https://www.datacamp.com/blog/best-ai-agents?utm_source=openai))

- **OpenAI Agents SDK**: A lightweight Python framework released in March 2025, focusing on creating multi-agent workflows with comprehensive tracing and guardrails. ([datacamp.com](https://www.datacamp.com/blog/best-ai-agents?utm_source=openai))

- **LangGraph**: An extension of LangChain, LangGraph allows developers to define agents as state machines, ideal for complex agent workflows requiring deterministic control, retries, or branching logic. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

- **Semantic Kernel**: Backed by Microsoft, Semantic Kernel is an enterprise-ready SDK for AI agent development, blending symbolic AI, LLMs, and business rules into modular, reusable agents. ([ment.tech](https://www.ment.tech/ai-agent-development-frameworks-every-business-should-know-in-2025/?utm_source=openai))

These frameworks are widely adopted for their flexibility, scalability, and support for complex AI agent development. 

Take a look at the trace

https://platform.openai.com/traces

## Our Second Agent: Planner Agent

Given a query, come up with 5 ideas for web searches that could be run.

Use Structured Outputs as our way to ensure the Agent provides what we need.

In [6]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 5


INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# We use Pydantic objects to describe the Schema of the output

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

# We pass in the Pydantic object to ensure the output follows the schema

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4.1-mini",
    output_type=WebSearchPlan,
)

In [7]:

message = "What are the most popular and successful AI Agent frameworks in June 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    pprint(result.final_output)

WebSearchPlan(searches=[WebSearchItem(reason='To identify the currently most popular AI agent frameworks as of the latest data in 2025.', query='most popular AI agent frameworks 2025'), WebSearchItem(reason='To find which AI agent frameworks are considered most successful in terms of adoption, features, and community support in 2025.', query='successful AI agent frameworks 2025'), WebSearchItem(reason='To gather expert reviews and comparisons of AI agent frameworks available in 2025 to evaluate their strengths and weaknesses.', query='AI agent frameworks comparison review 2025'), WebSearchItem(reason='To check recent industry reports and market analysis on AI agent frameworks adoption and trends for 2025.', query='AI agent frameworks market trends 2025'), WebSearchItem(reason='To explore developer community feedback and GitHub stats for various AI agent frameworks popular in 2025.', query='AI agent frameworks GitHub stars and developer reviews 2025')])


In [8]:
for search in result.final_output.searches:
    print("I will run this search:", search.query)
    print("For this reason:", search.reason)
    print("--------------------------------")
    

I will run this search: most popular AI agent frameworks 2025
For this reason: To identify the currently most popular AI agent frameworks as of the latest data in 2025.
--------------------------------
I will run this search: successful AI agent frameworks 2025
For this reason: To find which AI agent frameworks are considered most successful in terms of adoption, features, and community support in 2025.
--------------------------------
I will run this search: AI agent frameworks comparison review 2025
For this reason: To gather expert reviews and comparisons of AI agent frameworks available in 2025 to evaluate their strengths and weaknesses.
--------------------------------
I will run this search: AI agent frameworks market trends 2025
For this reason: To check recent industry reports and market analysis on AI agent frameworks adoption and trends for 2025.
--------------------------------
I will run this search: AI agent frameworks GitHub stars and developer reviews 2025
For this reaso

## Our Third Agent: Writer Agent

Take the results of internet searches and make a report

In [9]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

## Our Fourth Agent: push notification


In [10]:

@function_tool
def push(message: str):
    """Send a push notification with this brief message

    Args:
        message: The message to send to the user as a push notification.
    """
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    return {"status": "success"}

In [11]:
push.params_json_schema

{'properties': {'message': {'description': 'The message to send to the user as a push notification.',
   'title': 'Message',
   'type': 'string'}},
 'required': ['message'],
 'title': 'push_args',
 'type': 'object',
 'additionalProperties': False}

In [12]:
INSTRUCTIONS = """You are a member of a research team and will be provided with a short summary of a report.
When you receive the report summary, you send a push notification to the user using your tool, informing them that research is complete,
and including the report summary you receive"""


push_agent = Agent(
    name="Push agent",
    instructions=INSTRUCTIONS,
    tools=[push],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required")
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [13]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and send a push notification

In [14]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_push(report: ReportData):
    """ Use the push agent to send a notification to the user """
    print("Pushing...")
    await Runner.run(push_agent, report.short_summary)
    print("Push sent")
    return report

### Showtime!

In [15]:
query ="What are the most popular and successful AI Agent frameworks in May 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_push(report)  
    print("Hooray!")
display(Markdown(report.markdown_report))

Starting research...
Planning searches...
Will perform 5 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Pushing...
Push sent
Hooray!


# Report on AI Agent Frameworks Popularity and Success in May 2025

## Table of Contents
1. **Introduction**  
2. **Overview of AI Agent Frameworks in 2025**  
   1. LangChain  
   2. AutoGen  
   3. CrewAI  
   4. Semantic Kernel  
   5. LangGraph  
   6. LlamaIndex  
   7. OpenAI Agents SDK  
3. **Real-world Applications**  
4. **Market Trends and Projections**  
5. **Conclusion**  
6. **Follow-up Questions**  

---  

## 1. Introduction  
The field of artificial intelligence (AI) continues to innovate at a remarkable pace. In May 2025, AI agent frameworks have become integral to the development of intelligent systems, facilitating complex task execution, enhancing operational efficiency, and improving user experience across various sectors. This report reviews the most popular and successful AI agent frameworks as of May 2025, discusses their unique features, and outlines their applications and implications for the future.  

## 2. Overview of AI Agent Frameworks in 2025  
Numerous AI frameworks have emerged, each targeting specific aspects of AI development. Below are the leading frameworks:

### 2.1 LangChain  
LangChain has established itself as a versatile Python framework for building large language model (LLM) applications, including AI agents. Its modular architecture allows for the creation of complex workflows by integrating diverse tools, APIs, and memory management capabilities. This feature makes LangChain particularly adaptable for developers aiming to build customized AI applications. With a supportive community, it remains a primary choice for sophisticated AI development.  

### 2.2 AutoGen  
Developed by Microsoft, AutoGen emphasizes multi-agent orchestration, allowing AI agents to communicate naturally. This framework includes various built-in agents and a user-friendly graphical interface for prototype development. Particularly effective in domains requiring dynamic interactions among agents, AutoGen caters to applications ranging from automated customer service to collaborative research tasks.  

### 2.3 CrewAI  
CrewAI introduces a role-based design focusing on collaboration among AI agents. This lightweight Python framework is tailored for structured teamwork, making it ideal for scenarios like project management or marketing automation. Its user-friendly setup lowers the barrier to entry for teams wanting to implement AI-driven initiatives.  

### 2.4 Semantic Kernel  
Also from Microsoft, Semantic Kernel is an enterprise-ready SDK that integrates symbolic AI and LLMs. This framework is well-suited for applications requiring robust security and compliance, while supporting multiple programming languages including Python and C#. It encapsulates powerful orchestration capabilities suitable for enterprise contexts where integration of AI skills into existing applications is critical.  

### 2.5 LangGraph  
An extension of LangChain, LangGraph offers a graph-based approach to agent design, allowing developers to create stateful, controllable multi-agent systems. This innovative design facilitates complex workflows with conditional logic and state persistence, positioning it as a significant, emerging tool for constructing sophisticated AI applications.  

### 2.6 LlamaIndex  
LlamaIndex focuses on Retrieval-Augmented Generation (RAG), gaining attention for its capabilities in data indexing and context creation. This framework excels at supporting teams aiming to build AI agents requiring substantial data processing and contextual understanding. Its application areas span from content generation to contextually aware customer interactions.  

### 2.7 OpenAI Agents SDK  
The OpenAI Agents SDK serves as a lightweight solution for creating multi-agent workflows within the OpenAI ecosystem. The SDK's emphasis on tracing and implementing guardrails makes it suitable for applications that benefit from coordination among multiple agents. Its ease of use aligns well with Python developers seeking a straightforward entry into multi-agent systems.  

## 3. Real-world Applications  
The rapid advancement of AI agent frameworks has translated into significant real-world applications. For example:
- **Siemens AG** employs agentic AI for analyzing real-time sensor data from industrial machinery, enabling them to predict failure events and reduce unplanned downtime by 25%.
- **JPMorgan Chase** developed an AI agent, known as "LOXM," which autonomously executes high-frequency trading strategies, dynamically adjusting to market fluctuations faster than traditional human traders.  
- In healthcare, several hospital networks have integrated AI agents to manage scheduling and organize medical records, reportedly decreasing administrative workloads by around 30%.  
- A financial services firm utilized AI agents to expedite its loan approval processes, reducing the time taken from three days to just four hours, leading to a 40% cut in operational costs.

These applications illustrate how these AI frameworks enhance efficiency, reduce costs, and elevate customer satisfaction across various industries.  

## 4. Market Trends and Projections  
The AI agent framework sector is experiencing exponential growth. Projections suggest a compound annual growth rate (CAGR) of approximately 45% from 2025 to 2030. This expansion is prompted by increasing adoption of autonomous agents and multi-agent systems across sectors such as finance, healthcare, and manufacturing. Industry experts emphasize a trend toward developing more collaborative, intuitive agents capable of understanding and responding to complex social and operational dynamics.

Frameworks like LangChain and AutoGen are lauded for their approachability and innovative capabilities. In contrast, frameworks from larger companies, while robust, often come with steeper learning curves and stricter integration demands. The ongoing demand for custom AI solutions ensures that flexibility and adaptability will be vital attributes of future frameworks.  

## 5. Conclusion  
As we analyze the AI landscape in May 2025, it’s evident that frameworks such as LangChain, AutoGen, CrewAI, and Semantic Kernel are set to define the future trajectory of AI agent development. With their continuous evolution and user-friendly design, these frameworks empower a broad range of applications across industries, paving the way for more sophisticated, adaptive AI solutions that enhance productivity and streamline workflows.

## 6. Follow-up Questions  
- What are the specific use cases driving the adoption of AI agent frameworks in different sectors?  
- How do various frameworks compare in terms of scalability and performance in high-demand environments?  
- What future innovations can we anticipate in the AI agent framework space in the next 5 years?  
- What role does community feedback play in the development and evolution of these frameworks?  
- Are there potential ethical considerations or challenges associated with the increased reliance on AI agents in business operations?

### Take a look at the trace

https://platform.openai.com/traces